In [1]:
import torch, torchvision
import torch.nn.functional as ff
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch import optim
import numpy as np
import matplotlib.pyplot as plt
from time import time

In [2]:
#Configuration & Loading MNIST:
batch_size = 20

transform_src = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0,), (1,)),])    #zero mean, unit variance

training_set = datasets.MNIST('./data/', train=True, download=True, transform=transform_src)
training_loader = torch.utils.data.DataLoader(training_set, batch_size=batch_size, shuffle=True)

testing_set = datasets.MNIST('./data/', train=False, download=True, transform=transform_src)
testing_loader = torch.utils.data.DataLoader(testing_set, batch_size=batch_size, shuffle=True)

In [3]:
#Building the model:
class ConvNetwork(nn.Module):
    def __init__(self):
        super(ConvNetwork, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=10, kernel_size=3, stride=1, padding=1)

        self.conv3 = nn.Conv2d(in_channels=10, out_channels=100, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(in_channels=100, out_channels=125, kernel_size=3, stride=1, padding=1)
        
        self.maxpool = nn.MaxPool2d(2,2)

        self.fc1 = nn.Linear(784, 60)
        self.fc2 = nn.Linear(60, 10)
        
        self.relu = nn.ReLU()
        self.leaky_relu = nn.LeakyReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, img):
        x = self.leaky_relu(self.conv1(img))
        x = self.leaky_relu(self.conv2(x)) 
        x = self.maxpool(x)
        x = self.leaky_relu(self.conv3(x))
        x = self.leaky_relu(self.conv4(x))
        x = self.maxpool(x)
        x = img.view(x.size(0), -1)        #Flatten
        x = self.leaky_relu(self.fc1(x))
        x = self.leaky_relu(self.fc2(x))
        x = self.softmax(x)
        return x

net = ConvNetwork()

In [4]:
print(net)

ConvNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(6, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(10, 100, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(100, 125, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=784, out_features=60, bias=True)
  (fc2): Linear(in_features=60, out_features=10, bias=True)
  (relu): ReLU()
  (leaky_relu): LeakyReLU(negative_slope=0.01)
  (softmax): Softmax(dim=1)
)


In [ ]:
#Training the model:
cross_loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)
time0 = time()
epochs = 30
training_loss = []

for epoch in range(epochs):
    step_loss = []
    net.train()
    running_loss = 0
    for data in training_loader:
        x, y = data
        y_hot = ff.one_hot(y, num_classes=10) 
        optimizer.zero_grad()
        output = net(x)
        loss = cross_loss(output, y_hot.float())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()  
        step_loss.append(loss.item())
    else:
        print("Epoch {} - Training loss: {}".format(epoch, running_loss/len(training_loader)))
        training_loss.append(np.array(step_loss).mean())
print("\nTraining Time (in minutes) =",(time()-time0)/60)

In [ ]:
#Testing the model:
def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    time00 = time()
    loss = nn.CrossEntropyLoss()
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += loss(output, target).item()    #sum up batch loss
            pred = output.argmax(1, keepdim=True)        #get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
         correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    print("\nTesting Time (in seconds) =",(time()-time00))

In [ ]:
test(net, testing_loader)

In [ ]:
#Plotting:
plt.plot(training_loss, label='training_loss', color='r')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.xlim(0, 30)
plt.legend()
plt.grid(True)
plt.show